## Linear Optimization (pyomo vs. linopy)


### Exercise:


A post office requires a different number of full time employees on different days of the week: Each worker must work 5 consecutive days and then have 2 days off. Minimize the number of workers to be hired.

Sets:
$$D = \text{set of week days}$$
$$W = \text{set of consecutive working days}$$
Variables:
$$x \in \mathbb{Z}^+$$
Objective:
minimize the total number of employees
$$min_x\text{ z} = \sum_{d \in D} x_d$$
subject to:
$$\sum_{w \in W} x_{(d+w)} \mod 7 \geq \epsilon _d \quad \forall d \in D$$
$$\epsilon_d:\text{number of full time employees per day}$$  
with "number of full time employees per day":
<table style="margin-left: auto;margin-right: auto;">
  <tr>
    <td>Monday</td>
    <td>Tuesday</td>
    <td>Wednesday</td>
    <td>Thursday</td>
    <td>Friday</td>
    <td>Saturday</td>
    <td>Sunday</td>
  </tr>
  <tr>
    <td>17</td>
    <td>13</td>
    <td>15</td>
    <td>19</td>
    <td>14</td>
    <td>16</td>
    <td>11</td>
  </tr>
</table>


In [10]:
d = range(7)

def test(d: int):
    return [ (d + i)%7 for i in range(5)]

for day in d:
    print(f"{day}: {test(day)}")

0: [0, 1, 2, 3, 4]
1: [1, 2, 3, 4, 5]
2: [2, 3, 4, 5, 6]
3: [3, 4, 5, 6, 0]
4: [4, 5, 6, 0, 1]
5: [5, 6, 0, 1, 2]
6: [6, 0, 1, 2, 3]


## Pyomo Solution

In [3]:
import pyomo.environ as pyomo
model = pyomo.ConcreteModel()

model.d = pyomo.Set(initialize = range(7), doc='days of the week, starting at monday')
model.wd = pyomo.Set(initialize = range(5), doc='consecutive working days')

model.x = pyomo.Var(model.d, within=pyomo.NonNegativeIntegers, doc='number of starting workers per day' )
model.epsilon = pyomo.Param(
    model.d,
    initialize={day: employees for day, employees in zip(model.d, [17,13,15,19,14,16,11])},
    doc='number of full time employees per day'
)

model.total_number_employees = pyomo.Objective(expr = sum(model.x[d] for d in model.d), sense=pyomo.minimize)

def rule1(model, d):
    return sum(model.x[(d+ working_day)%7]  for working_day in model.wd)  >= model.epsilon[d]

model.employees_day_constraint = pyomo.Constraint(model.d, rule=rule1, doc="full time employees per day requirement")

optimizer = pyomo.SolverFactory('appsi_highs')
result = optimizer.solve(model)

result.write()
print("\n# " + "="*50)
print(f"# Number of overall employees: {model.total_number_employees()}")
print("# " + "="*50 +"\n")
model.pprint()


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: 23.0
  Upper bound: 23.0
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 0
  Sense: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Termination message: TerminationCondition.optimal
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of s

# Linopy Solution

In [144]:
import linopy
import xarray as xr
import pandas as pd


In [145]:
model = linopy.Model()

## Sets
# the days of the week, starting at monday
d = pd.RangeIndex(7) #, name='days')
# consecutive working days
wd = pd.RangeIndex(5)

## Variables
#   x(d)    number of starting workers per day
#   z       total number of employees to be hired
x = model.add_variables(lower=0, coords=[d],name='x', integer=True)


In [146]:

## Parameters
# number of full time employees per day
epsilon = xr.DataArray([17, 13, 15, 19, 14, 16, 11], coords=[d], name="epsilon")

In [147]:
## Constraints

def con_lhs(model, d):
    return sum(x[(d+ working_day)%7] for working_day in wd)

con = model.linexpr(con_lhs, [d]) >= epsilon


In [148]:
model.add_constraints(con, name='full time employee requirement')

Constraint `full time employee requirement` (dim_0: 7):
-------------------------------------------------------
[0]: +1 x[0] + 1 x[1] + 1 x[2] + 1 x[3] + 1 x[4] ≥ 17.0
[1]: +1 x[1] + 1 x[2] + 1 x[3] + 1 x[4] + 1 x[5] ≥ 13.0
[2]: +1 x[2] + 1 x[3] + 1 x[4] + 1 x[5] + 1 x[6] ≥ 15.0
[3]: +1 x[3] + 1 x[4] + 1 x[5] + 1 x[6] + 1 x[0] ≥ 19.0
[4]: +1 x[4] + 1 x[5] + 1 x[6] + 1 x[0] + 1 x[1] ≥ 14.0
[5]: +1 x[5] + 1 x[6] + 1 x[0] + 1 x[1] + 1 x[2] ≥ 16.0
[6]: +1 x[6] + 1 x[0] + 1 x[1] + 1 x[2] + 1 x[3] ≥ 11.0

In [149]:
model.constraints

linopy.model.Constraints
------------------------
 * full time employee requirement (dim_0)

In [150]:
print(linopy.available_solvers)

['highs']


In [151]:
objective = x.sum()
model.add_objective(objective, sense="min")

model.solve()


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
7 rows, 7 cols, 35 nonzeros
7 rows, 7 cols, 35 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   7 rows
   7 cols (0 binary, 7 integer, 0 implied int., 0 continuous)
   35 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               25               100.00%        0      0      0         0     0.0s
 R       0       0         0   0.00%   22.33333333     24                 6.94%        0      0      0         7     0.0s
 H       0       0         0   0.00%   23         

('ok', 'optimal')

In [152]:
x

Variable (dim_0: 7)
-------------------
[0]: x[0] ∈ Z ⋂ [0,...,inf]
[1]: x[1] ∈ Z ⋂ [0,...,inf]
[2]: x[2] ∈ Z ⋂ [0,...,inf]
[3]: x[3] ∈ Z ⋂ [0,...,inf]
[4]: x[4] ∈ Z ⋂ [0,...,inf]
[5]: x[5] ∈ Z ⋂ [0,...,inf]
[6]: x[6] ∈ Z ⋂ [0,...,inf]

In [153]:
model.solution

<xarray.Dataset> Size: 112B
Dimensions:  (dim_0: 7)
Coordinates:
  * dim_0    (dim_0) int64 56B 0 1 2 3 4 5 6
Data variables:
    x        (dim_0) float64 56B 6.0 0.0 4.0 0.0 7.0 5.0 1.0

In [ ]:
Pyomo:
https://github.com/jckantor/ND-Pyomo-Cookbook/tree/main
https://github.com/Pyomo/PyomoGallery/wiki